In [1]:
import os
import sys
from keras.models import Model
from keras.layers import concatenate
from keras.utils.data_utils import get_file
from keras.optimizers import RMSprop
from data_generator.config import mpii_sp_dataconf
from data_generator.mpii import MpiiSinglePerson
from data_generator.loader import BatchLoader
from models.losses import pose_regression_loss
from models import reception
from models.mpii_tools import eval_singleperson_pckh
from models.ntu_pose_tool import NTU_POSE_EvalCallback
from keras.callbacks import LearningRateScheduler
from models.callbacks import SaveModel
from data_generator.io_operate import *
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from data_generator.pose import *
from data_generator.ntu import Ntu
from data_generator.config import ntu_pe_dataconf
import keras

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
num_blocks = 4
batch_size = 24
input_shape = mpii_sp_dataconf.input_shape
num_joints = 20
dataset_path1 = './data/mpii'
dataset_path2 = './data/ntu'
TEST_MODE = 0
TRAIN_MODE = 1
VALID_MODE = 2

In [3]:
mpii = MpiiSinglePerson(dataset_path1, dataconf=mpii_sp_dataconf,poselayout = pa20j3d)
ntu = Ntu(dataset_path2, dataconf = ntu_pe_dataconf,topology = 'frame', use_gt_bbox = True, clip_size=20)

merge_tr = BatchLoader([mpii,ntu], ['frame'], ['pose'], TRAIN_MODE,batch_size=batch_size, \
                       num_predictions=num_blocks, shuffle=True)

merge_val = BatchLoader(ntu, x_dictkeys=['frame'],y_dictkeys=['pose', 'afmat', 'ntuaction','rootz'], mode=VALID_MODE,
        batch_size=7000, num_predictions=1,shuffle=False)

[x_val], [p_val, afmat_val, action_val,rootz] = merge_val[0]

with tf.device("/cpu:0"):
    single_model = reception.build(input_shape, num_joints, dim=3,num_blocks=num_blocks,\
                                   ksize=(5, 5), depth_maps = 8)
multi_model = keras.utils.multi_gpu_model(single_model,gpus = 2) 

logdir = './ntu_pose'

eval_callback = NTU_POSE_EvalCallback(x_val, p_val, afmat_val, rootz,action_val,\
                eval_model=multi_model, batch_size=2, logdir=logdir)
single_model.summary()

Max/Min number of frames: 245/43
Number of videos: [0, 2639, 2346]


/home/xd133/Graduation Project/Codes/Burt Fan/data_generator/ntu.py:196: RuntimeWarning: Mean of empty slice
  rootz = np.nanmean(pose[:, 0, 2])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
Stem (Model)                    (None, 32, 32, 576)  1039488     input_1[0][0]                    
__________________________________________________________________________________________________
rBlock1 (Model)                 (None, 32, 32, 576)  1312128     Stem[1][0]                       
__________________________________________________________________________________________________
SepConv1 (Model)                (None, 32, 32, 576)  347904      rBlock1[1][0]                    
__________________________________________________________________________________________________
RegMap1 (M

In [4]:
loss = pose_regression_loss('l1l2bincross', 0.01)
multi_model.compile(loss=loss, optimizer=RMSprop())


def lr_scheduler(epoch, lr):

    if epoch in [80, 100]:
        newlr = 0.2*lr
        printcn(WARNING, 'lr_scheduler: lr %g -> %g @ %d' % (lr, newlr, epoch))
    else:
        newlr = lr
        printcn(OKBLUE, 'lr_scheduler: lr %g @ %d' % (newlr, epoch))

    return newlr

weights_file = 'weights_ntu_pos_{epoch:03d}.h5'
callbacks = []
callbacks.append(SaveModel(os.path.join(logdir,weights_file),model_to_save = single_model))
callbacks.append(LearningRateScheduler(lr_scheduler))
callbacks.append(eval_callback)

In [5]:
multi_model.fit_generator(merge_tr,
        epochs=120,
        callbacks=callbacks,
        workers=4,
        initial_epoch=0)

Epoch 1/120
lr_scheduler: lr 0.001 @ 0
 41/929 [>.............................] - ETA: 14:16 - loss: 0.3527 - concatenate_6_loss: 0.0873 - concatenate_8_loss: 0.0882 - concatenate_10_loss: 0.0886 - concatenate_12_loss: 0.0887

/home/xd133/Graduation Project/Codes/Burt Fan/data_generator/ntu.py:289: RuntimeWarning: invalid value encountered in greater
  v = np.apply_along_axis(lambda x: 1 if x.all() else 0,axis=2, arr=(p > 0))


929/929 [==============================] - 675s 727ms/step - loss: 0.2410 - concatenate_6_loss: 0.0611 - concatenate_8_loss: 0.0600 - concatenate_10_loss: 0.0599 - concatenate_12_loss: 0.0600
Saving model @epoch=00001 to ./ntu_pose/weights_ntu_pos_001.h5
7000/7000 [==============================] - 74s 11ms/step
Avg. mm. error: 101.0 96.5 100.4 98.6
drink water: 102.6
eat meal/snack: 85.1
brushing teeth: 94.3
brushing hair: 91.8
drop: 82.8
pickup: 136.3
throw: 88.0
sitting down: 112.4
standing up (from sitting position): 98.7
clapping: 84.9
reading: 85.6
writing: 82.9
tear up paper: 87.4
wear jacket: 128.2
take off jacket: 125.3
wear a shoe: 129.9
take off a shoe: 107.5
wear on glasses: 90.5
take off glasses: 89.0
put on a hat/cap: 105.0
take off a hat/cap: 101.6
cheer up: 104.1
hand waving: 109.4
kicking something: 93.1
put something inside pocket / take out something from pocket: 66.1
hopping (one foot jumping): 83.2
jump up: 80.8
make a phone call/answer phone: 64.2
playing with pho

put the palms together: 73.2
cross hands in front (say stop): 90.8
sneeze/cough: 75.7
staggering: 94.8
falling: 139.1
touch head (headache): 104.4
touch chest (stomachache/heart pain): 70.1
touch back (backache): 83.6
touch neck (neckache): 76.4
nausea or vomiting condition: 118.1
use a fan (with hand or paper)/feeling warm: 81.6
Final averaged error (mm): 92.854
Best score is 92.9 at epoch 4
Epoch 5/120
lr_scheduler: lr 0.001 @ 4
929/929 [==============================] - 658s 708ms/step - loss: 0.1668 - concatenate_6_loss: 0.0447 - concatenate_8_loss: 0.0412 - concatenate_10_loss: 0.0405 - concatenate_12_loss: 0.0404
Saving model @epoch=00005 to ./ntu_pose/weights_ntu_pos_005.h5
7000/7000 [==============================] - 72s 10ms/step
Avg. mm. error: 118.1 114.9 105.7 104.5
drink water: 116.3
eat meal/snack: 103.0
brushing teeth: 106.2
brushing hair: 110.2
drop: 91.4
pickup: 117.6
throw: 97.6
sitting down: 112.8
standing up (from sitting position): 97.8
clapping: 105.7
reading: 95.

put something inside pocket / take out something from pocket: 64.6
hopping (one foot jumping): 72.1
jump up: 70.1
make a phone call/answer phone: 62.7
playing with phone/tablet: 61.0
typing on a keyboard: 77.4
pointing to something with finger: 100.0
taking a selfie: 106.0
check time (from watch): 78.7
rub two hands together: 73.4
nod head/bow: 70.6
shake head: 63.8
wipe face: 65.9
salute: 69.1
put the palms together: 65.9
cross hands in front (say stop): 83.5
sneeze/cough: 68.4
staggering: 89.4
falling: 132.2
touch head (headache): 98.9
touch chest (stomachache/heart pain): 67.3
touch back (backache): 73.1
touch neck (neckache): 65.9
nausea or vomiting condition: 113.6
use a fan (with hand or paper)/feeling warm: 78.8
Final averaged error (mm): 85.018
Best score is 85.0 at epoch 8
Epoch 9/120
lr_scheduler: lr 0.001 @ 8
929/929 [==============================] - 654s 704ms/step - loss: 0.1513 - concatenate_6_loss: 0.0410 - concatenate_8_loss: 0.0373 - concatenate_10_loss: 0.0365 - conc

...

nausea or vomiting condition: 103.4
use a fan (with hand or paper)/feeling warm: 64.1
Final averaged error (mm): 77.978
Best score is 72.0 at epoch 55
Epoch 106/120
lr_scheduler: lr 4e-05 @ 105
929/929 [==============================] - 652s 702ms/step - loss: 0.0946 - concatenate_6_loss: 0.0283 - concatenate_8_loss: 0.0233 - concatenate_10_loss: 0.0217 - concatenate_12_loss: 0.0213
Saving model @epoch=00106 to ./ntu_pose/weights_ntu_pos_106.h5
7000/7000 [==============================] - 71s 10ms/step
Avg. mm. error: 90.6 103.5 83.3 79.1
drink water: 87.0
eat meal/snack: 72.4
brushing teeth: 64.7
brushing hair: 83.8
drop: 65.0
pickup: 104.2
throw: 71.4
sitting down: 89.2
standing up (from sitting position): 68.0
clapping: 70.1
reading: 71.9
writing: 67.0
tear up paper: 81.2
wear jacket: 107.8
take off jacket: 107.3
wear a shoe: 110.8
take off a shoe: 84.4
wear on glasses: 78.7
take off glasses: 78.7
put on a hat/cap: 89.2
take off a hat/cap: 87.2
cheer up: 87.0
hand waving: 82.5
kicki

pointing to something with finger: 86.4
taking a selfie: 81.9
check time (from watch): 73.8
rub two hands together: 64.6
nod head/bow: 66.3
shake head: 55.1
wipe face: 63.3
salute: 53.2
put the palms together: 58.5
cross hands in front (say stop): 67.2
sneeze/cough: 55.9
staggering: 75.9
falling: 119.8
touch head (headache): 82.1
touch chest (stomachache/heart pain): 56.3
touch back (backache): 60.6
touch neck (neckache): 58.1
nausea or vomiting condition: 98.6
use a fan (with hand or paper)/feeling warm: 58.2
Final averaged error (mm): 77.082
Best score is 72.0 at epoch 55
Epoch 110/120
lr_scheduler: lr 4e-05 @ 109
929/929 [==============================] - 653s 703ms/step - loss: 0.0940 - concatenate_6_loss: 0.0281 - concatenate_8_loss: 0.0231 - concatenate_10_loss: 0.0216 - concatenate_12_loss: 0.0212
Saving model @epoch=00110 to ./ntu_pose/weights_ntu_pos_110.h5
7000/7000 [==============================] - 72s 10ms/step
Avg. mm. error: 88.6 80.0 77.9 82.1
drink water: 94.7
eat meal

wear a shoe: 135.1
take off a shoe: 115.5
wear on glasses: 100.5
take off glasses: 98.1
put on a hat/cap: 119.0
take off a hat/cap: 113.0
cheer up: 104.9
hand waving: 109.3
kicking something: 104.6
put something inside pocket / take out something from pocket: 88.7
hopping (one foot jumping): 94.5
jump up: 94.2
make a phone call/answer phone: 84.9
playing with phone/tablet: 83.0
typing on a keyboard: 108.6
pointing to something with finger: 125.0
taking a selfie: 123.1
check time (from watch): 112.5
rub two hands together: 98.7
nod head/bow: 111.8
shake head: 86.7
wipe face: 98.0
salute: 94.6
put the palms together: 91.3
cross hands in front (say stop): 117.8
sneeze/cough: 95.5
staggering: 105.0
falling: 156.1
touch head (headache): 123.8
touch chest (stomachache/heart pain): 85.2
touch back (backache): 92.4
touch neck (neckache): 95.0
nausea or vomiting condition: 136.2
use a fan (with hand or paper)/feeling warm: 97.5
Final averaged error (mm): 108.018
Best score is 72.0 at epoch 55
E

7000/7000 [==============================] - 71s 10ms/step
Avg. mm. error: 103.3 111.6 85.5 126.8
drink water: 104.9
eat meal/snack: 81.6
brushing teeth: 86.7
brushing hair: 94.6
drop: 70.4
pickup: 107.3
throw: 69.3
sitting down: 104.1
standing up (from sitting position): 81.2
clapping: 86.0
reading: 77.6
writing: 75.0
tear up paper: 81.7
wear jacket: 113.4
take off jacket: 122.2
wear a shoe: 117.3
take off a shoe: 89.4
wear on glasses: 92.8
take off glasses: 90.9
put on a hat/cap: 99.4
take off a hat/cap: 96.3
cheer up: 92.8
hand waving: 90.5
kicking something: 74.5
put something inside pocket / take out something from pocket: 51.5
hopping (one foot jumping): 65.3
jump up: 60.3
make a phone call/answer phone: 56.7
playing with phone/tablet: 54.1
typing on a keyboard: 82.8
pointing to something with finger: 93.0
taking a selfie: 90.2
check time (from watch): 83.6
rub two hands together: 78.3
nod head/bow: 73.7
shake head: 63.6
wipe face: 71.6
salute: 60.0
put the palms together: 66.8
c

touch back (backache): 72.8
touch neck (neckache): 67.9
nausea or vomiting condition: 112.1
use a fan (with hand or paper)/feeling warm: 77.0
Final averaged error (mm): 88.203
Best score is 72.0 at epoch 55
